In [1]:
import pandas as pd
import numpy as np

import rasterio
from skimage.transform import resize
from skimage.transform import rotate
import os

import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.model_selection import train_test_split

from datetime import timedelta
from skimage.draw import polygon
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

#### Import Yield Data

In [2]:
from utils import process_yield_data
from pathlib import Path
YIELD_DATA_PATH = Path("./combined_yield_data.csv")
yield_data_weekly = process_yield_data(YIELD_DATA_PATH)

            Volume (Pounds)  Cumulative Volumne (Pounds)  Pounds/Acre
Date                                                                 
2012-01-02          23400.0                      23400.0          2.0
2012-01-03          26064.0                      49464.0          3.0
2012-01-04          32382.0                      81846.0          3.0
2012-01-05          69804.0                     151650.0          7.0
2012-01-06          18000.0                     169650.0          2.0

Number of Yield Data Points:  3970

Column Names: Index(['Volume (Pounds)', 'Cumulative Volumne (Pounds)', 'Pounds/Acre'], dtype='object')
Number of Yield Data Points: 2879
Yield data with time features:
            Volume (Pounds)  Cumulative Volumne (Pounds)  Pounds/Acre  \
Date                                                                    
2012-03-04         525753.0                    1785843.0    18.333333   
2012-03-11        2949534.0                    4735377.0    51.666667   
2012-03-18   

#### Define the Model

In [3]:
target_shape = (512, 512)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [4]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(CNNFeatureExtractor, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flattened_size = self._get_conv_output((1, *target_shape))
        self.fc1 = nn.Linear(self.flattened_size, 512)

    def _get_conv_output(self, shape):
        x = torch.rand(1, *shape)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        n_size = x.view(1, -1).size(1)
        return n_size

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout(x)
        x = x.view(-1, self.flattened_size)
        x = F.relu(self.fc1(x))
        return x
    
class HybridModel(nn.Module):
    def __init__(self, cnn_feature_extractor, lstm_hidden_size=64, lstm_layers=1):
        super(HybridModel, self).__init__()
        self.cnn = cnn_feature_extractor
        self.lstm = nn.LSTM(input_size=512, hidden_size=lstm_hidden_size, num_layers=lstm_layers, batch_first=True)
        self.fc1 = nn.Linear(lstm_hidden_size + 6, 64)
        self.fc2 = nn.Linear(64, target_shape[0] * target_shape[1])  # Predict a value per pixel
        self.target_shape = target_shape

    def forward(self, x, time_features):
        batch_size, time_steps, C, H, W = x.size()
        c_in = x.view(batch_size * time_steps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, time_steps, -1)
        r_out, (h_n, c_n) = self.lstm(r_in)
        r_out = r_out[:, -1, :]
        x = torch.cat((r_out, time_features), dim=1)  # Concatenate LSTM output with time features
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.view(batch_size, *self.target_shape)  # Reshape to the target shape
        return x

#### Initialize Function

In [5]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

# Instantiate model with weight decay regularization
cnn_feature_extractor = CNNFeatureExtractor()
model = HybridModel(cnn_feature_extractor)
model.apply(weights_init)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

#### Training loop with early stopping

In [6]:
best_loss = float('inf')
patience = 3
trigger_times = 0
epochs = 50

# for epoch in range(epochs):
#     running_loss = 0.0
#     model.train()
#     for i, (inputs, labels, time_features) in enumerate(tqdm(train_loader)):
#         if device != "cpu":
#             inputs, labels, time_features = inputs.to(device), labels.to(device), time_features.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs, time_features)
#         labels = labels.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     epoch_loss = running_loss / len(train_loader)
#     scheduler.step()
#     print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

#     # Early stopping
#     if epoch_loss < best_loss:
#         best_loss = epoch_loss
#         trigger_times = 0
#         torch.save(model.state_dict(), 'best_hybrid_model.pth')  # Save best model
#     else:
#         trigger_times += 1
#         if trigger_times >= patience:
#             print("Early stopping!")
#             break


# def train_model(model, optimizer, scheduler, criterion, train_loader, val_loader, epochs, device, patience=3):
#     best_loss = float('inf')
#     trigger_times = 0
    
#     for epoch in range(epochs):
#         model.train()
#         running_loss = 0.0
#         for inputs, labels, time_features in tqdm(train_loader):
#             inputs, labels, time_features = inputs.to(device), labels.to(device), time_features.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs, time_features)
#             labels_expanded = labels.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
#             loss = criterion(outputs, labels_expanded)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()
        
#         epoch_loss = running_loss / len(train_loader)
#         scheduler.step()
        
#         val_loss = 0.0
#         model.eval()
#         with torch.no_grad():
#             for inputs, labels, time_features in val_loader:
#                 inputs, labels, time_features = inputs.to(device), labels.to(device), time_features.to(device)
#                 outputs = model(inputs, time_features)
#                 labels_expanded = labels.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
#                 loss = criterion(outputs, labels_expanded)
#                 val_loss += loss.item()
        
#         val_loss /= len(val_loader)
#         print(f'Epoch {epoch+1}, Training Loss: {epoch_loss}, Validation Loss: {val_loss}')
        
#         # Early stopping
#         if val_loss < best_loss:
#             best_loss = val_loss
#             trigger_times = 0
#             torch.save(model.state_dict(), 'best_hybrid_model.pth')  # Save best model
#         else:
#             trigger_times += 1
#             if trigger_times >= patience:
#                 print("Early stopping!")
#                 break

# # Training and validation
# # model = HybridModel(CNNFeatureExtractor()).to(device)
# # criterion = nn.MSELoss()
# # optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
# # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# # train_model(model, optimizer, scheduler, criterion, train_loader, val_loader, epochs=50, device=device)

# DIEGO: is everything above out-of-date and only the functions below are relevant?

### Functions for prediction

In [7]:
from inference_utils import (
    preprocess_image,
    compute_mean_std,
    load_evi_data_and_prepare_features,
    find_closest_date,
    find_closest_date_in_df,
    mask_evi_data,
    predict,
    predict_weekly_yield
)


evi_data_dir = "./landsat_evi_monterey_masked"
evi_data_dict, time_features, mean, std  = load_evi_data_and_prepare_features(evi_data_dir, time_index=yield_data_weekly.index, target_shape=target_shape)

In [8]:
# #TODO: Convert the pixel area to physical area (may require a dynamic conversion factor depending on the zoom from the UI)
# #TODO: Pass the target date (for prediction) from the UI to this script
                                                                                                                                                                                            
# # Example polygon coordinates (we need to replace these with actual coordinates from your UI) - this one is just a box
# polygon_coords = np.array([
#     [100, 100],
#     [100, 200],
#     [200, 200],
#     [200, 100],
#     [100, 100]
# ])

# # Calculate the area based on the polygon coordinates
# polygon = Polygon(polygon_coords)
# polygon_area = polygon.area 

# # Convert polygon area from pixel units to acres (using the LandSat Conversion Factor)
# conversion_factor = 30  # 1 pixel = 30m^2
# polygon_area_acres = polygon_area * conversion_factor

# # Start date for predictions
# start_date = pd.to_datetime("2022-07-01") # This is an example date; we need to take this from the UI

# # Load and preprocess the EVI data
# time_index = yield_data_weekly.index
# evi_data_dict, time_features_list, mean, std = load_evi_data_and_prepare_features(evi_data_dir, time_index, target_shape)

# # Generate weekly predictions
# dates, predicted_yields = predict_weekly_yield(evi_data_dict, yield_data_weekly, start_date, polygon_area_acres, mean, std, target_shape, model, device)

# # Convert predictions to a numpy array
# predicted_yields = np.array(predicted_yields).flatten()

### Plot the predictions

In [9]:
# # Plot the predicted yields as a trend line
# plt.figure(figsize=(10, 6))
# plt.plot(dates, predicted_yields, marker='o', linestyle='-', color='b')
# plt.xlabel('Date')
# plt.ylabel('Predicted Yield (Pounds)')
# plt.title(f'Predicted Weekly Yield for Selected Polygon Area (Starting {start_date})')
# plt.grid(True)
# plt.show()

### Model Evaluation (Basic Evaluation Metrics)

In [10]:
# # Function to evaluate the model on the test set
# def evaluate_model(model, test_loader, mean, std, target_shape, device):
#     model.eval()
#     all_true = []
#     all_pred = []
    
#     with torch.no_grad():
#         for inputs, labels, time_features in test_loader:
#             if device != "cpu":
#                 inputs, labels, time_features = inputs.to(device), labels.to(device), time_features.to(device)
            
#             outputs = model(inputs, time_features)
#             labels = labels.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
            
#             all_true.append(labels.cpu().numpy())
#             all_pred.append(outputs.cpu().numpy())
    
#     all_true = np.concatenate(all_true).flatten()
#     all_pred = np.concatenate(all_pred).flatten()
    
#     mse = mean_squared_error(all_true, all_pred)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(all_true, all_pred)
#     r2 = r2_score(all_true, all_pred)
    
#     return mse, rmse, mae, r2

# # Evaluate the model on the test set
# mse, rmse, mae, r2 = evaluate_model(model, test_loader, mean, std, target_shape, device)

# print(f"Mean Squared Error (MSE): {mse}")
# print(f"Root Mean Squared Error (RMSE): {rmse}")
# print(f"Mean Absolute Error (MAE): {mae}")
# print(f"R-squared (R²): {r2}")

### Model Evaluation (Cross Validation)

In [18]:
# Cross-validation
tscv = TimeSeriesSplit(n_splits=5)

mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

def train_and_evaluate(model, train_loader, val_loader, optimizer, scheduler, criterion, epochs, device):
    best_loss = float('inf')
    patience = 3
    trigger_times = 0
    
    for epoch in range(epochs):
        running_loss = 0.0
        model.train()
        for inputs, labels, time_features in tqdm(train_loader):
            inputs, labels, time_features = inputs.to(device), labels.to(device), time_features.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, time_features)
            labels = labels.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        scheduler.step()
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')

        # Early stopping
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            trigger_times = 0
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping!")
                break

    # Evaluate on validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels, time_features in val_loader:
            inputs, labels, time_features = inputs.to(device), labels.to(device), time_features.to(device)
            outputs = model(inputs, time_features)
            labels = labels.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    print(f'Validation Loss: {val_loss}')
    return val_loss

for fold, (train_index, val_index) in enumerate(tscv.split(yield_data_weekly)):
    print(f"Fold {fold + 1}")
    print(f"       Train Index = {train_index[0]}, ..., {train_index[-1]} len={len(train_index)}")
    print(f"       Valid Index = {val_index[0]}, ..., {val_index[-1]} len={len(val_index)}")

    train_dates = yield_data_weekly.index[train_index].intersection(evi_data_dict.keys())
    val_dates = yield_data_weekly.index[val_index].intersection(evi_data_dict.keys())
    print(f"       Train Dates = {train_dates[0]}, ..., {train_dates[-1]} len={len(train_dates)}")
    print(f"       Valid Dates = {val_dates[0]}, ..., {val_dates[-1]} len={len(val_dates)}")

    train_index = yield_data_weekly.index.get_indexer(train_dates)
    val_index = yield_data_weekly.index.get_indexer(val_dates)
    print(f"  (get)Train Index = {train_index[0]}, ..., {train_index[-1]} len={len(train_index)}")
    print(f"  (get)Valid Index = {val_index[0]}, ..., {val_index[-1]} len={len(val_index)}")

    evi_train = np.array([evi_data_dict[date] for date in train_dates])
    evi_val = np.array([evi_data_dict[date] for date in val_dates])
    print(f"  evi_train.shape  = {evi_train.shape}")
    print(f"  evi_val.shape    = {evi_val.shape}")

    time_features_train = yield_data_weekly.loc[train_dates][['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'Volume (Pounds)', 'Cumulative Volumne (Pounds)']].values
    time_features_val = yield_data_weekly.loc[val_dates][['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'Volume (Pounds)', 'Cumulative Volumne (Pounds)']].values
    print(f"  time_features_train.shape  = {time_features_train.shape}")
    print(f"  time_features_val.shape    = {time_features_val.shape}")
    labels_train = yield_data_weekly.loc[train_dates]['Volume (Pounds)'].values
    labels_val = yield_data_weekly.loc[val_dates]['Volume (Pounds)'].values
    print(f"  labels_train.shape  = {labels_train.shape}")
    print(f"  labels_val.shape    = {labels_val.shape}")

    evi_train = torch.tensor(evi_train, dtype=torch.float32).unsqueeze(1).unsqueeze(2).to(device)
    evi_val = torch.tensor(evi_val, dtype=torch.float32).unsqueeze(1).unsqueeze(2).to(device)
    print(f"  (torch)evi_train.shape  = {evi_train.shape}")
    print(f"  (torch)evi_val.shape    = {evi_val.shape}")
    time_features_train = torch.tensor(time_features_train, dtype=torch.float32).to(device)
    time_features_val = torch.tensor(time_features_val, dtype=torch.float32).to(device)
    labels_train = torch.tensor(labels_train, dtype=torch.float32).to(device)
    labels_val = torch.tensor(labels_val, dtype=torch.float32).to(device)

    train_loader = DataLoader(list(zip(evi_train, labels_train, time_features_train)), batch_size=2, shuffle=True)
    val_loader = DataLoader(list(zip(evi_val, labels_val, time_features_val)), batch_size=2, shuffle=False)

    model = HybridModel(CNNFeatureExtractor())
    model.apply(weights_init)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    criterion = nn.MSELoss()

    val_loss = train_and_evaluate(model, train_loader, val_loader, optimizer, scheduler, criterion, epochs, device)
    
    model.eval()
    with torch.no_grad():
        outputs_val = model(evi_val, time_features_val)

    outputs_val_flat = outputs_val.cpu().numpy().flatten()
    labels_val_expanded = labels_val.unsqueeze(1).unsqueeze(2).expand(-1, target_shape[0], target_shape[1])
    labels_val_flat = labels_val_expanded.cpu().numpy().flatten()

    mse = mean_squared_error(labels_val_flat, outputs_val_flat)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(labels_val_flat, outputs_val_flat)
    r2 = r2_score(labels_val_flat, outputs_val_flat)

    mse_scores.append(mse)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

print(f"Average MSE: {np.mean(mse_scores)}")
print(f"Average RMSE: {np.mean(rmse_scores)}")
print(f"Average MAE: {np.mean(mae_scores)}")
print(f"Average R-squared: {np.mean(r2_scores)}")



Fold 1
       Train Index = 0, ..., 110 len=111
       Valid Index = 111, ..., 216 len=106
       Train Dates = 2014-02-23 00:00:00, ..., 2014-02-23 00:00:00 len=1
       Valid Dates = 2016-04-17 00:00:00, ..., 2016-04-17 00:00:00 len=1
  (get)Train Index = 103, ..., 103 len=1
  (get)Valid Index = 215, ..., 215 len=1
  evi_train.shape  = (1, 512, 512)
  evi_val.shape    = (1, 512, 512)
  time_features_train.shape  = (1, 6)
  time_features_val.shape    = (1, 6)
  labels_train.shape  = (1,)
  labels_val.shape    = (1,)
  (torch)evi_train.shape  = torch.Size([1, 1, 1, 512, 512])
  (torch)evi_val.shape    = torch.Size([1, 1, 1, 512, 512])


100%|██████████| 1/1 [00:00<00:00, 20.94it/s]


Epoch 1, Loss: 0.13545666635036469


100%|██████████| 1/1 [00:00<00:00, 24.23it/s]


Epoch 2, Loss: 0.1347959339618683


...

Epoch 18, Loss: 0.09234229226907094


100%|██████████| 3/3 [00:00<00:00, 23.18it/s]


Epoch 19, Loss: 0.10780388365189235


100%|██████████| 3/3 [00:00<00:00, 23.15it/s]


Epoch 20, Loss: 0.10758685568968455
Early stopping!
Validation Loss: 0.14833451341837645
Average MSE: 0.17643526196479797
Average RMSE: 0.41369152069091797
Average MAE: 0.3471130132675171
Average R-squared: -39942233052372.36


In [12]:
# save model to file
torch.save(model.state_dict(), "diego-bad-model.pt")
del model

## Inference

In [13]:
# load in model from file
inf_model_weights = torch.load("diego-bad-model.pt", weights_only=True)
inf_model = HybridModel(CNNFeatureExtractor())
inf_model.load_state_dict(inf_model_weights)
inf_model.to(device)
inf_model.eval()
inf_output = inf_model(evi_val, time_features_val)

print(f"{evi_val.shape = }")
print(f"{time_features_val.shape = }")
print(f"{inf_output.shape = }")

evi_val.shape = torch.Size([8, 1, 1, 512, 512])
time_features_val.shape = torch.Size([8, 6])
inf_output.shape = torch.Size([8, 512, 512])


What are the shapes above? Specifically the input?

[8, 1, 1, 512, 512]

[n_samples, ?, ?, width, height]


## Train on full dataset

In [14]:

# define the model 
model = HybridModel(CNNFeatureExtractor())
model.apply(weights_init)
model.to(device)

# define training criteria
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = nn.MSELoss()

In [15]:
# create full datasets for training
# make validation a lil bit smaller so we get a "better" model for inference?

# make our own train_index and val_index for now
n_train_samples = int(len(yield_data_weekly) * 0.9)
n_val_samples = len(yield_data_weekly) - n_train_samples
indices = np.arange(0, len(yield_data_weekly), 1, dtype=int)
train_index = indices[0:n_train_samples]
val_index = indices[n_train_samples:]
print(f"n_train_samples({len(train_index)}) + n_val_samples({len(val_index)}) == len(yield_weekly_data)({len(yield_data_weekly)}) = {len(train_index)+len(val_index) == len(yield_data_weekly)}")


# print(f"Train Indexes = {train_index}")
# print(f"Valid Indexes = {val_index}")

train_dates = yield_data_weekly.index[train_index].intersection(evi_data_dict.keys())
# val_dates = yield_data_weekly.index[val_index].intersection(evi_data_dict.keys())
# print(f"       Train Dates = {train_dates[0]}, ..., {train_dates[-1]} len={len(train_dates)}")
# print(f"       Valid Dates = {val_dates[0]}, ..., {val_dates[-1]} len={len(val_dates)}")

# train_index = yield_data_weekly.index.get_indexer(train_dates)
# val_index = yield_data_weekly.index.get_indexer(val_dates)
# print(f"  (get)Train Index = {train_index[0]}, ..., {train_index[-1]} len={len(train_index)}")
# print(f"  (get)Valid Index = {val_index[0]}, ..., {val_index[-1]} len={len(val_index)}")

# evi_train = np.array([evi_data_dict[date] for date in train_dates])
# evi_val = np.array([evi_data_dict[date] for date in val_dates])
# print(f"  evi_train.shape  = {evi_train.shape}")
# print(f"  evi_val.shape    = {evi_val.shape}")

# time_features_train = yield_data_weekly.loc[train_dates][['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'Volume (Pounds)', 'Cumulative Volumne (Pounds)']].values
# time_features_val = yield_data_weekly.loc[val_dates][['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'Volume (Pounds)', 'Cumulative Volumne (Pounds)']].values
# labels_train = yield_data_weekly.loc[train_dates]['Volume (Pounds)'].values
# labels_val = yield_data_weekly.loc[val_dates]['Volume (Pounds)'].values

# evi_train = torch.tensor(evi_train, dtype=torch.float32).unsqueeze(1).unsqueeze(2).to(device)
# time_features_train = torch.tensor(time_features_train, dtype=torch.float32).to(device)
# time_features_val = torch.tensor(time_features_val, dtype=torch.float32).to(device)
# labels_train = torch.tensor(labels_train, dtype=torch.float32).to(device)
# labels_val = torch.tensor(labels_val, dtype=torch.float32).to(device)

# train_loader = DataLoader(list(zip(evi_train, labels_train, time_features_train)), batch_size=2, shuffle=True)
# val_loader = DataLoader(list(zip(evi_val, labels_val, time_features_val)), batch_size=2, shuffle=False)


n_train_samples(576) + n_val_samples(65) == len(yield_weekly_data)(641) = True


In [16]:
evi_data_dict.keys()

dict_keys([Timestamp('2014-02-23 00:00:00'), Timestamp('2016-04-17 00:00:00'), Timestamp('2017-10-29 00:00:00'), Timestamp('2020-04-12 00:00:00'), Timestamp('2022-04-10 00:00:00'), Timestamp('2022-07-31 00:00:00'), Timestamp('2022-09-25 00:00:00'), Timestamp('2022-11-20 00:00:00'), Timestamp('2023-03-12 00:00:00'), Timestamp('2023-05-07 00:00:00'), Timestamp('2023-07-02 00:00:00'), Timestamp('2023-10-22 00:00:00'), Timestamp('2023-12-17 00:00:00')])

In [17]:
yield_data_weekly.index[train_index]

DatetimeIndex(['2012-03-04', '2012-03-11', '2012-03-18', '2012-03-25',
               '2012-04-01', '2012-04-08', '2012-04-15', '2012-04-22',
               '2012-04-29', '2012-05-06',
               ...
               '2023-01-08', '2023-01-15', '2023-01-22', '2023-01-29',
               '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-26',
               '2023-03-05', '2023-03-12'],
              dtype='datetime64[ns]', name='Date', length=576, freq=None)